In [38]:
import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / 'src'
DATA_RAW_DIR = PROJECT_ROOT / 'data' / 'raw' / 'psychiatric-motor-activity-dataset'
sys.path.append(str(SRC_DIR))
from core.pma_base import PMActivity
from core.pma_base import PMADataADHDKaggle
import matplotlib.pyplot as plt
import pandas as pd

In [29]:
FEATURES = DATA_RAW_DIR / 'features.csv'
df=pd.read_csv(FEATURES)
df.head(20)

,user,mean,sd,pctZeros,median,q25,q75,class
0,control_1,185.568056,346.555786,38.680556,37.5,0.0,201.00,control
1,control_1,225.981250,384.378661,35.833333,53.0,0.0,248.00,control
2,control_1,169.663194,304.317895,45.625000,18.0,0.0,194.00,control
3,control_1,183.690972,277.006551,36.111111,106.0,0.0,184.00,control
4,control_1,347.075000,236.845868,2.361111,347.0,264.0,347.00,control
5,control_1,194.423611,356.423708,43.750000,20.0,0.0,194.00,control
6,control_1,310.715278,450.937392,35.763889,93.0,0.0,495.00,control
7,control_2,271.193056,360.034647,29.513889,136.0,0.0,394.00,control
8,control_2,462.234028,534.682651,27.569444,283.0,0.0,757.25,control
9,control_2,457.170139,591.562696,33.888889,156.0,0.0,801.00,control


In [36]:
import statistics as stat
from datetime import datetime
DATAFILE = DATA_RAW_DIR / 'control' / 'control_1.csv'
df_tmp = pd.read_csv(DATAFILE)
dates = df_tmp.loc[:,'date'].unique()
for d in ['2003-03-19']: #dates:
    mask= df_tmp.loc[:,'date'] == d
    if mask.sum() == 1440:
        activity = df_tmp.loc[mask,'activity']
        q25, median, q75 = stat.quantiles(activity, n=4)
        print(activity.describe())
        print(q25, median, q75)
        mean = stat.mean(activity)
        sd = stat.stdev(activity)
        print(mean,sd)
        pctZeros = 100 * (activity == 0).mean()
        print(pctZeros)

df_features_selfmade = pd.DataFrame(columns=['user', 'mean', 'sd', 'pctZeros', 
                            'median', 'q25', 'q75', 'class', 'date', 'weekday'])

diags  = ['adhd', 'clinical', 'control', 'depression', 'schizophrenia']
pat_lists = {}
max_npat = 0
for diag in diags:
    df = pd.read_csv(DATA_RAW_DIR / f"{diag}-info.csv")
    pat_lists[diag] = list(df.loc[:,'number'])
    print(pat_lists[diag])
    npat = len(pat_lists[diag])
    max_npat = max(max_npat, npat)

for idiag in range(len(diags)):
    diag = diags[idiag]
    pat_list = pat_lists[diag]
    print(pat_list)
    pat_id = pat_list[0]
    for ipat_id in range(len(pat_list)):
        pat_id = pat_list[ipat_id]
        DATAFILE = DATA_RAW_DIR / diag / f'{pat_id}.csv'
        df_tmp = pd.read_csv(DATAFILE)
        dates = df_tmp.loc[:,'date'].unique()
        for d in dates:
            mask= df_tmp.loc[:,'date'] == d
            if mask.sum() == 1440:
                activity = df_tmp.loc[mask,'activity']
                q25, median, q75 = stat.quantiles(activity, n=4)
                mean = stat.mean(activity)
                sd = stat.stdev(activity)
                pctZeros = 100 * (activity == 0).mean()
                weekday = datetime.strptime(str(d),'%Y-%m-%d').strftime('%A')
                df_features_selfmade.loc[len(df_features_selfmade)] = \
                    [pat_id, mean, sd, pctZeros, median, q25, q75, diag, d, weekday]


df_features_selfmade.head()


count    1440.000000
mean      185.568056
std       346.555786
min         0.000000
25%         0.000000
50%        37.500000
75%       201.000000
max      3097.000000
Name: activity, dtype: float64
0.0 37.5 201.0
185.56805555555556 346.5557857963215
38.68055555555556
['adhd_1', 'adhd_2', 'adhd_3', 'adhd_5', 'adhd_8', 'adhd_11', 'adhd_13', 'adhd_14', 'adhd_19', 'adhd_20', 'adhd_21', 'adhd_26', 'adhd_41', 'adhd_43', 'adhd_45', 'adhd_48', 'adhd_49', 'adhd_51', 'adhd_52', 'adhd_55', 'adhd_57', 'adhd_58', 'adhd_59', 'adhd_60', 'adhd_63', 'adhd_65', 'adhd_67', 'adhd_68', 'adhd_71', 'adhd_72', 'adhd_74', 'adhd_75', 'adhd_77', 'adhd_83', 'adhd_84', 'adhd_85', 'adhd_87', 'adhd_88', 'adhd_91', 'adhd_92', 'adhd_94', 'adhd_95', 'adhd_97', 'adhd_98', 'adhd_108']
['clinical_7', 'clinical_9', 'clinical_10', 'clinical_15', 'clinical_18', 'clinical_22', 'clinical_23', 'clinical_24', 'clinical_27', 'clinical_30', 'clinical_31', 'clinical_32', 'clinical_33', 'clinical_34', 'clinical_35', 'clinical_36', 

,user,mean,sd,pctZeros,median,q25,q75,class,date,weekday
0,adhd_1,54.134028,101.653198,46.180556,5.0,0.0,65.0,adhd,2009-02-24,Tuesday
1,adhd_1,9.389583,41.273697,88.541667,0.0,0.0,0.0,adhd,2009-02-25,Wednesday
2,adhd_1,29.054861,78.145117,72.152778,0.0,0.0,5.0,adhd,2009-02-26,Thursday
3,adhd_1,99.136806,159.225772,46.319444,3.0,0.0,147.0,adhd,2009-02-27,Friday
4,adhd_1,41.240972,85.954516,54.236111,0.0,0.0,48.0,adhd,2009-02-28,Saturday


In [39]:
pat_id='control_1'
mask = df_features_selfmade.loc[:,'user'] == pat_id
print(df_features_selfmade.loc[mask,:])
print(df.head(20))

          user        mean          sd    pctZeros  median    q25    q75  \
502  control_1  185.568056  346.555786   38.680556    37.5    0.0  201.0   
503  control_1  225.981250  384.378661   35.833333    53.0    0.0  248.0   
504  control_1  169.663194  304.317895   45.625000    18.0    0.0  194.0   
505  control_1  183.690972  277.006551   36.111111   106.0    0.0  184.0   
506  control_1  347.075000  236.845868    2.361111   347.0  264.0  347.0   
507  control_1  194.423611  356.423708   43.750000    20.0    0.0  194.0   
508  control_1  310.715278  450.937392   35.763889    93.0    0.0  495.0   
509  control_1  199.518750  365.761889   48.680556     3.0    0.0  273.0   
510  control_1  183.131944  319.061505   40.208333    27.0    0.0  201.0   
511  control_1  218.872222  391.002171   44.791667    22.0    0.0  223.0   
512  control_1  276.368750  387.813892   36.458333   106.0    0.0  421.0   
513  control_1  232.002778  435.231500   41.111111    37.0    0.0  248.0   
514  control

In [6]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier

model_tree = DecisionTreeClassifier(class_weight = 'balanced', 
                                    random_state = 42,
                                    min_samples_split=20,
                                    max_depth = 4)
model_forest = RandomForestClassifier(class_weight = 'balanced', 
                                     random_state = 42,
                                     max_depth = 4 )

features = df.drop(['user','class'], axis=1)
target_adhd = df.loc[:,'class'] == 'schizophrenia'
target_adhd.sum()

np.int64(263)